# Lecture8 Recurrent Neural Networks

这次课推导RNN，介绍各种训练技巧和拓展变种。梯度消失的推导很详细，用Python演示很直观，也给出了用裁剪防止梯度爆炸的直观解释。笔记里还补充了用于机器翻译时的5项改进。cliping.pngen_decoder.png

## 传统语言模型

语言模型就是计算一个单词序列（句子）的概率$P(w_{1},…,w_{m})$的模型。听上去很简单，做起来很难；听上去没什么用处，但用处非常多。比如在机器翻译中，判断译文序列中一种词序的自然程度高于另一种，判断一种用词选择优于另一种。

为了简化问题，必须引入马尔科夫假设，句子的概率通常是通过待预测单词之前长度为n的窗口建立条件概率来预测：

![屏幕快照 2018-08-28 下午3.31.23.png](https://i.loli.net/2018/08/28/5b84fb5da862c.png)

为了估计此条件概率，常用极大似然估计，比如对于BiGram和TriGram模型，有：

![屏幕快照 2018-08-28 下午3.31.43.png](https://i.loli.net/2018/08/28/5b84fb5dab6dd.png)

在数据量足够的情况下，n-gram中的n越大，模型效果越好。但实际上，数据量总是不如人意，这时候一些平滑方法就不可或缺。另外，这些ngram可能会占用上G的内存，在最新的研究中，一个1260亿的语料在140G内存的单机上花了2.8天才得到结果。

Bengio et al提出了第一个大规模深度学习自然语言处理模型，只不过是用前n个单词的词向量来做同样的事情（上文建模）而已，其网络结构如下：

![006Fmjmcly1fgttmr9ufjj30kb0i5wgm.jpg](https://i.loli.net/2018/08/28/5b84fb5e16992.jpg)

公式如下：

![屏幕快照 2018-08-28 下午3.33.26.png](https://i.loli.net/2018/08/28/5b84fb5da9f10.png)

这里$W^{(3)}x+b^{(3)}$就是前n个单词词向量的线性运算，虽然这种模型名字里有“Neural”，但依然属于传统模型。

## Recurrent Neural Networks

新的语言模型是利用RNN对序列建模，复用不同时刻的线性非线性单元及权值，理论上之前所有的单词都会影响到预测单词。

![006Fmjmcly1fgtta8g6cmj30mt0d0400.jpg](https://i.loli.net/2018/08/28/5b84fff857fdc.jpg)

所需内存只与词表大小成正比，不取决于序列长度。

给定一个词向量序列：$ x_{1},...,x_{t−1},x_{t},x_{t+1},...x_{T} $，在每个时间点上都有隐藏层的特征表示：

$$h_{t}=\sigma (W^{(hh)}h_{t−1}+W^{(hx)}x_{t})$$

![屏幕快照 2018-08-28 下午3.50.40.png](https://i.loli.net/2018/08/28/5b84fff8bf4ec.png)

未unroll的网络：

![006Fmjmcly1fgtx8rq2ouj30ju0dfwfi.jpg](https://i.loli.net/2018/08/28/5b84fff7e6019.jpg)

等效于

![006Fmjmcly1fgtx9lzahdj30c108ygm1.jpg](https://i.loli.net/2018/08/28/5b84fff7e4a57.jpg)

### 损失函数

![屏幕快照 2018-08-28 下午4.03.15.png](https://i.loli.net/2018/08/28/5b850314d1416.png)

### 训练RNN很难

观察句子1：

> "Jane walked into the room. John walked in too. Jane said hi to ___"

以及句子2：

> "Jane walked into the room. John walked in too. It was late in the day, and everyone was walking home after a long day at work. Jane said hi to ___"

人类可以轻松地在两个空中填入“John”这个答案，但RNN却很难做对第二个。这是因为在前向传播的时候，前面的$x$反复乘上$W$，导致对后面的影响很小。

反向传播时也是如此。

![屏幕快照 2018-08-28 下午4.05.18.png](https://i.loli.net/2018/08/28/5b850314f2165.png)

![屏幕快照 2018-08-28 下午4.05.43.png](https://i.loli.net/2018/08/28/5b850315094d9.png)

而大于1时，浮点数运算会产生溢出（NaN），一般可以很快发现。这叫做梯度爆炸。小于1，或者下溢出并不产生异常，难以发现，但会显著降低模型对较远单词的记忆效果，这叫做梯度消失。

### 梯度消失实例

有个[IPython Notebook](http://cs224d.stanford.edu/notebooks/vanishing_grad_example.html)专门演示梯度消失，对于如下数据：

![006Fmjmcly1fgu4mum47qj30gt0dbmyh.jpg](https://i.loli.net/2018/08/28/5b85031488fc4.jpg)

学习非线性的决策边界：

![006Fmjmcly1fgub149f16j30gm0d6q45.jpg](https://i.loli.net/2018/08/28/5b8503148bfeb.jpg)

用经典的三层网络结构，得到蓝色的第一层梯度的长度和绿色的第二层梯度的长度，可视化：

sigmoid激活函数下：

![006Fmjmcly1fgu4oxwhzpj30h60dldg1.jpg](https://i.loli.net/2018/08/28/5b85041d092e2.jpg)

ReLU激活函数下：

![006Fmjmcly1fgu4pbjc7rj30h60dlt8w.jpg](https://i.loli.net/2018/08/28/5b85041d0a3bb.jpg)

在这个例子的反向传播中，相邻两层梯度是近乎减半地减小。

### 防止梯度爆炸

一种暴力的方法是，当梯度的长度大于某个阈值的时候，将其缩放到某个阈值。虽然在数学上非常丑陋，但实践效果挺好。

其直观解释是，在一个只有一个隐藏节点的网络中，损失函数和权值w偏置b构成error surface，其中有一堵墙：

![006Fmjmcly1fgu80lzfw9j30pr0g70yp.jpg](https://i.loli.net/2018/08/28/5b85041d75a1f.jpg)

每次迭代梯度本来是正常的，一次一小步，但遇到这堵墙之后突然梯度爆炸到非常大，可能指向一个莫名其妙的地方（实线长箭头）。但缩放之后，能够把这种误导控制在可接受的范围内（虚线短箭头）。

但这种trick无法推广到梯度消失，因为你不想设置一个最低值硬性规定之前的单词都相同重要地影响当前单词。

### 减缓梯度消失

与其随机初始化参数矩阵，不如初始化为单位矩阵。这样初始效果就是上下文向量和词向量的平均。然后用ReLU激活函数。这样可以在step多了之后，依然使得模型可训练。

![006Fmjmcly1fgu8faifyuj30my0jctei.jpg](https://i.loli.net/2018/08/28/5b85041d4273e.jpg)

困惑度结果

![006Fmjmcly1fgu8jbejq7j30y00dialo.jpg](https://i.loli.net/2018/08/28/5b85041d80515.jpg)

### 问题：softmax太大了

词表太大的话，softmax很费力。一个技巧是，先预测词语的分类（比如按词频分），然后在分类中预测词语。分类越多，困惑度越小，但速度越慢。所以存在一个平衡点：

![006Fmjmcly1fgu8nn5srqj30s20kegyf.jpg](https://i.loli.net/2018/08/28/5b85041da62c5.jpg)

### 最后的实现技巧

记录每个t的误差不要丢，反向传播的时候将其累加起来。

## 序列模型的应用

可以把每个词分类到NER、实体级别的情感分析（饭菜味道不错，但环境不太卫生）、意见表达。

其中，意见挖掘任务就是将每个词语归类为：

DSE：直接主观描述（明确表达观点等）

ESE：间接主观描述（间接地表达情感等）

语料标注采用经典的BIO标注：

![006Fmjmcly1fgu8zv8hgfj31a20i8q8o.jpg](https://i.loli.net/2018/08/28/5b8506215a7f0.jpg)

实现这个任务的朴素网络结构就是一个裸的RNN：

![006Fmjmcly1fgu95qcl7tj30ww0d0t9r.jpg](https://i.loli.net/2018/08/28/5b850620b8466.jpg)

但是这个网络无法利用当前词语的下文辅助分类决策，解决方法是使用一些更复杂的RNN变种。

### Bidirectional RNNs

![屏幕快照 2018-08-28 下午4.19.49.png](https://i.loli.net/2018/08/28/5b850621961aa.png)

这里箭头表示从左到右或从右到左前向传播，对于每个时刻t的预测，都需要来自双向的特征向量，拼接后进行分类。箭头虽然不同，但参数还是同一套参数（有些地方是两套参数[1]	G. Lample, M. Ballesteros, S. Subramanian, K. Kawakami, and C. Dyer, “Neural Architectures for Named Entity Recognition.,” HLT-NAACL, 2016.）。

### Deep Bidirectional RNNs

理解了上图之后，再加几个层，每个时刻不但接受上个时刻的特征向量，还接受来自下层的特征表示：

![屏幕快照 2018-08-28 下午4.20.27.png](https://i.loli.net/2018/08/28/5b850621c684d.png)

### 评测

评测方法是标准的F1（因为标签样本不均衡），在不同规模的语料上试验不同层数的影响：

![006Fmjmcly1fgu9skeeejj31a80nqtba.jpg](https://i.loli.net/2018/08/28/5b850a9e30541.jpg)

可见层数不是越多越好。

### 应用：RNN机器翻译模型

传统机器翻译模型在不同的阶段用到大量不同的机器学习算法，这里讨论用RNN统一整个流水线。

比如将3个单词的德语翻译为2个单词的英语，用到如下RNN：

![屏幕快照 2018-08-28 下午4.23.56.png](https://i.loli.net/2018/08/28/5b850aa077bf2.png)

但预测准确率不理想，有如下拓展方法：

1. encoder和decoder使用不同的权值矩阵。也就是上述两个$W^{(hh)}$不再相同。
2. decoder中的隐藏层的输入来自3个方面：

    - 前一个时刻的隐藏层
    - encoder的最后一个隐藏层$c=hT$
    - 前一个预测结果 $ŷ_{t−1}$

    这样导致decoder函数变为：$$h_{t}= \Phi (h_{t−1},c,y_{t−1})$$

    ![006Fmjmcly1fguakyetk3j30gf0g6q48.jpg](https://i.loli.net/2018/08/28/5b850a9d828dc.jpg)

3. 使用深度RNN
4. 使用 bi-directional encoder
5. 不再用$A B C→X Y$作为训练实例，而是逆转原文词序：$C B A→X Y$。因为$A$更可能翻译为$X$，而梯度消失导致$A$无法影响输出，倒过来$A$离输出近一些。



## 回顾

RNN是最好的DeepNLP模型之一

因为梯度消失和梯度爆炸，训练很难

可以用很多技巧来训练

下次课将介绍更强大的RNN拓展：LSTM和GRU


